In [ ]:
# imports

import requests
from bs4 import BeautifulSoup #to scrape website
from IPython.display import Markdown, display
from openai import OpenAI #to connect to ai modal

In [ ]:
# Use open ai library to connect to locally running ollama api
# Here it is - see the base_url

openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [ ]:
# Let's try a  simple prompt

message = "Hello, Llama! This is my first ever message to you! Hi!"
response = openai.chat.completions.create(model="llama3.2", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

In [ ]:
#Now we will build a sample application to summarize any given website

headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [ ]:
website = Website("https://medium.com/@snehabagri.90/choosing-kafka-wisely-17768a27c616")
print(website.title)
print(website.text)

In [ ]:
# Define our system prompt - that tells them what task they are performing and what tone they should use "

system_prompt = "You are a content manager that analyzes the main content of a website \
and produces a concise summary, omitting any navigation or menu-related text."

In [ ]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [ ]:
print(user_prompt_for(website))

In [ ]:
# OpenAI expects data in a specific format

messages = [
    {"role": "system", "content": "You are an AI engineer"},
    {"role": "user", "content": "Explain LLM to a kid?"}
]

In [ ]:
# To give you a preview -- calling OpenAI with system and user messages:

response = openai.chat.completions.create(model="llama3.2", messages=messages)
print(response.choices[0].message.content)

In [ ]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [ ]:
messages_for(website)

In [ ]:
# And now: call the OpenAI API with model llama

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "llama3.2",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [ ]:
summary=summarize("https://medium.com/@snehabagri.90/choosing-kafka-wisely-17768a27c616")
print(summary)

In [ ]:
Markdown(summary)